In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet, VGG16
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import cv2
import matplotlib.pyplot as plt
from scipy.integrate import simps
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from tensorflow.keras.utils import plot_model
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
from mtcnn.mtcnn import MTCNN 
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
import visualkeras
import matplotlib.pyplot as plt
from mtcnn import MTCNN  
from matplotlib.patches import Rectangle, Circle

2024-05-26 02:26:55.268405: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-26 02:26:55.268455: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-26 02:26:55.268491: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-26 02:26:55.276502: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
path="/home/gourav/Desktop/vss/capstone/best_model.h5"
img_size=224
model=tf.keras.models.load_model(path)
detector = MTCNN()

2024-05-26 02:27:00.619822: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-26 02:27:00.625472: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-26 02:27:00.625776: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    
    if not ret:
        print("Error: Could not read frame.")
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    detections = detector.detect_faces(rgb_frame)
    
    eyes_roi = None
    for detection in detections:
        x, y, w, h = detection['box']
        confidence = detection['confidence']
        keypoints = detection['keypoints']
        
        if confidence > 0.90: 

            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

            left_eye = keypoints['left_eye']
            right_eye = keypoints['right_eye']
            
            ex1, ey1 = left_eye
            ex2, ey2 = right_eye
            eye_x1 = min(ex1, ex2) - w // 10
            eye_y1 = min(ey1, ey2) - h // 10
            eye_x2 = max(ex1, ex2) + w // 10
            eye_y2 = max(ey1, ey2) + h // 10
            
            eyes_roi = frame[eye_y1:eye_y2, eye_x1:eye_x2]

            cv2.rectangle(frame, (eye_x1, eye_y1), (eye_x2, eye_y2), (255, 0, 0), 2)
    
    if eyes_roi is not None:
        try:

            final_image = cv2.resize(eyes_roi, (img_size, img_size))
            final_image = np.array(final_image).reshape(1, img_size, img_size, 3)  
            final_image = final_image / 255.0

            # Predict using the model
            prediction = model.predict(final_image)

            # Display prediction
            if prediction > 0.5:
                cv2.putText(frame, "Open Eyes", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)   
            else:
                cv2.putText(frame, "Closed Eyes", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
        except Exception as e:
            print(f"Error processing eyes_roi: {e}")

    # Display the frame
    cv2.imshow("Driver Drowsiness Detection", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


2024-05-26 02:27:04.484946: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700


1/1 [==============================] - 0s 15ms/step


In [ ]:
# image=cv2.imread(r'/home/gourav/Desktop/vss/capstone/dataset/no_yawn/3.jpg')
# image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# image=cv2.resize(image,(img_size,img_size))
# image=np.array(image).reshape(1,img_size,img_size,3)# (one image 224,224,3)
# image=image/255.0
# image.shape
# # if positive open and if negative closed
# prediction=model.predict(image)
# print(prediction)